# O que é?

se trata de um serviço de infraestrutura como codigo (IaC)

a vantagem de IaC é o controle de modo versionado e claro de como estão sendo criados, usados e alterados recursos, além do fato de permitir que mais de um usuário consiga lidar com uma arquitetura cloud, por exemplo, dando manutenção e assistência

# Vantagens

+ controle do ambiente

+ define o ambiente de modo determinístico, isto é, os recursos escritos irão refletir a estrutura do ambiente diretamente. assim, no caso de, por exemplo, aumentar uma unidade de EC2 fazemos relativo a 0 unidades e não à quantidade existente atualmente no ambiente (como é feito em Ansible)

# Desvantagens

+ por depender de organização de pastas e arquivos o gerenciamento feito em equipes deve ser sempre alinhado para não haver conflitos de escrita de codigo e alterações inesperadas na arquitetura

# Estrutura

## Componentes de uma infraestrutura terraform

+ environment/provider.tf

    + backend

+ environment/.tfstate

+ environment/.tfvars

+ environment/variables.tf

+ environment/main.tf

+ modules/

+ modules/output.tf

### provider.tf

local onde providers serão definidos e salvos

um provider se trata do responsável a **interagir com a API do serviço desejado**, no nosso caso AWS, permitindo realizar operações de **CRUD** (create, read, update e delete) sobre ele e seus recursos na linguagem HCL

um exemplo de definição de provider é o abaixo

```terraform
terraform {
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 5.0"
    }
  }
}

# Configure the AWS Provider
provider "aws" {
  region = "us-east-1"
}
```

nesse trecho de codigo podemos ressaltar alguns pontos bem importantes como por exemplo: 

+ o provider "hashicorp/aws" foi criado usando o nome "aws"

+ podemos fazer controle da versão usada como por exemplo >, >=, <, <=, = e ~> (sendo o ultimo significando que somente atualizações que não impactem a maior hierarquia de versões, nesse caso a major, sejam usadas, desse modo 5.1, 5.2, etc serão usadas sem problemas)

+ em seguida temos a configuração do provider com por exemplo a região desejada que ele opere, cada particularidade é definida pelo provider em sua documentação, não seguindo um padrão bem definido de nomenclatura de variaveis de provider para provider

#### aws provider

para o provider da aws em particular podemos especificar os seguintes argumentos

```terraform
provider "aws" {
  region                                     string /* pode ser encontrado de modo nativo pelo terraform em $HOME/.aws/config ou $HOME/.aws/credentials */
  access_key                                 string  /* pode ser encontrado de modo nativo pelo terraform em $HOME/.aws/config ou $HOME/.aws/credentials */
  secret_key                                 string /* pode ser encontrado de modo nativo pelo terraform em $HOME/.aws/config ou $HOME/.aws/credentials */
  profile                                    string /* pode ser encontrado de modo nativo pelo terraform em $HOME/.aws/config ou $HOME/.aws/credentials */
  shared_config_files                        list /* pode ser encontrado de modo nativo pelo terraform em $HOME/.aws/config ou $HOME/.aws/credentials */
  shared_credentials_files                   list  /* pode ser encontrado de modo nativo pelo terraform em $HOME/.aws/config ou $HOME/.aws/credentials */
  assume_role                                map[string]string
  assume_role_with_web_identity              map[string]string
}
```

+ profile (por padrão é default) usado para especificar um perfil nomeado

+ shared_config_files e shared_credentials_files podem ser encontrados nas variaveis de ambiente `AWS_CONFIG_FILE` e `AWS_SHARED_CREDENTIALS_FILE` em situações onde valores nao sejam passados de modo explícito

+ usando `assume_role` podemos fazer com que nosso provider da aws assuma um role para realizar as operações necessárias dentro do ambiente. isto pode ser feito de dois modos:

  + usando um único role:

  ```terraform
  provider "aws" {
    assume_role {
      role_arn     = "arn:aws:iam::123456789012:role/ROLE_NAME"
      session_name = "SESSION_NAME"
      external_id  = "EXTERNAL_ID"
    }
  }
  ```

  + definindo mais de um role:

  ```terraform
  provider "aws" {
    assume_role {
      role_arn = "arn:aws:iam::123456789012:role/INITIAL_ROLE_NAME"
    }
    assume_role {
      role_arn = "arn:aws:iam::123456789012:role/FINAL_ROLE_NAME"
    }
  }
  ```

+ o uso de `assume_role_with_web_identity` é feito quando possuimos uma credencial web e um ARN de role, desse modo o provider irá assumir as politicas desta role

```terraform
provider "aws" {
  assume_role_with_web_identity {
    role_arn                = "arn:aws:iam::123456789012:role/ROLE_NAME"
    session_name            = "SESSION_NAME"
    web_identity_token_file = "/Users/tf_user/secrets/web-identity-token"
  }
}
```

**mais detalhes sobre os argumentos do aws provider podem ser encontrados em https://registry.terraform.io/providers/hashicorp/aws/latest/docs**

### .tfstate

### .tfvars

### main.tf

### variables.tf

### modules

### output.tf

## Componentes de um arquivo

um arquivo terraform (.tf) é composto de blocks podendo eles serem dos seguintes tipos:

+ resource

+ data

+ locals

+ module

### resource

blocos como este sao usados na definição de recursos a serem criados no terraform

eles possuem como estrutura a seguinte:

```terraform
resource "<provider>_<recurso>" "<nome do recurso>"  {
        ...
}
```

podemos referenciar um recurso passando parametros dele para outro recurso usando referencia implicita, isto é, quando fazemos uso da notação abaixo para "linkar" dois recursos sem declarar explicitamente (como feito quando usado depends_on)

```terraform
resource "aws_instance" "ec2" {
        ...
}

resource "aws_s3_bucket" "bucket" {
        x = aws_instance.ec2.name
}
```

quando feita dependência implícita como no caso acima, ao executar `terraform apply` ou `terraform destroy`

### data

sao um tipos de resource que realiza somente a leitura de outros recursos, armazenando seu valor e permitindo a reutilização

### locals

sao variaveis que são acessiveis somente entro do arquivo tf onde são criadas, evitando repetição desnecessária e seguindo DRY (don't repeat yourself)

### module